# Setup

In [ ]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install --quiet -U --no-cache-dir gdown --pre
# !pip install --quiet nlp-id
# !pip install --quiet emoji
# !pip install --quiet imblearn
# !pip install --quiet gensim

In [ ]:
# from nlp_id.stopword import StopWord
# from nlp_id.postag import PosTag
# from nlp_id.lemmatizer import Lemmatizer

# import emoji
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='cp437')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/extracted_test_data.csv', encoding='cp437')
train_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/extracted_train_data.csv', encoding='cp437')

kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='cp437')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='cp437')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='cp437')

In [ ]:
HS_features = [
    'cebong',
    '2019gantipresiden',
    'Exclamation',      # Is there an exclammation?
    'jokowi',
    'prabowo',
    'ahok',
    'lengser',
    'Length',           # Length of tweet
    'Abusive',          # Is there any word that is in the abusive lexicon?
    'budaya',
    'tolol',
    'goblok',
    'Uppercase',        # How many uppercase letters?
    'bubar',
    'dungu',
    'Words',            # Amount of words in tweet
    'kampret',
    'bani',
    'dasar',
    'kafir',
    'otak',
    'katolik',
    'tipu',
    'bom',
    'biadab',
    'ekonomi',
    'kristen',
    'pki',
    'pilkada',
    'sontoloyo'
    ]

A_features = [
    'Abusive',          # Is there any word that is in the abusive lexicon?
    'cebong',
    'Length',           # Length of tweet
    'Words',            # Amount of words in tweet
    'goblok',
    'kampret',
    'tolol',
    'anjir',
    'kontol',
    'bajing',
    'anjing',
    'gubernur',
    'dungu',
    'memek',
    'jokowi',
    'otak',
    'kristen',
    'banci',
    'kunyuk',
    'ngentot',
    'bodoh',
    'presiden',
    'asing',
    'setan',
    'babi',
    'bani',
    'budaya',
    'berengsek',
    'bacot',
    'bego'
    ]

In [ ]:
bias = potensi_bias['0'].values

In [ ]:
HS_nbf = []
A_nbf = []

for hsf in HS_features:
  if hsf not in bias:
    HS_nbf.append(hsf)

for af in A_features:
  if af not in bias:
    A_nbf.append(af)

for col in train_data.columns:
  if col in bias:
    train_data.drop(col, inplace=True, axis=1)
    test_data.drop(col, inplace=True, axis=1)

In [ ]:
# remove bias features

In [ ]:
X_train = train_data.loc[:, train_data.columns != 'Tweet']
X_train = X_train.loc[:, X_train.columns != 'HS']
X_train = X_train.loc[:, X_train.columns != 'A']
Y_train_HS = train_data.loc[:, train_data.columns == 'HS']
Y_train_A = train_data.loc[:, train_data.columns == 'A']
Y_train = pd.DataFrame(Y_train_HS['HS'], columns=["HS"])
Y_train['A'] = Y_train_A['A']


X_test = test_data.loc[:, test_data.columns != 'Tweet']
X_test = X_test.loc[:, X_test.columns != 'HS']
X_test = X_test.loc[:, X_test.columns != 'A']
Y_test_HS = test_data.loc[:, test_data.columns == 'HS']
Y_test_A = test_data.loc[:, test_data.columns == 'A']
Y_test = pd.DataFrame(Y_test_HS['HS'], columns=["HS"])
Y_test['A'] = Y_test_A['A']

In [ ]:
X_train_HS = X_train.copy()
for col in X_train_HS:
  if col not in HS_nbf:
    X_train_HS.drop(col, inplace=True, axis=1)

X_test_HS = X_test.copy()
for col in X_test_HS:
  if col not in HS_nbf:
    X_test_HS.drop(col, inplace=True, axis=1)

X_train_A = X_train.copy()
for col in X_train_A:
  if col not in A_features:
    X_train_A.drop(col, inplace=True, axis=1)

X_test_A = X_test.copy()
for col in X_test_A:
  if col not in A_features:
    X_test_A.drop(col, inplace=True, axis=1)

# Binary Classification

## Hate Speech

### Logistic Regression

#### HS features

In [ ]:
# all parameters not specified are set to their defaults
lr_HS = LogisticRegression()

lr_HS.fit(X_train_HS, Y_train_HS)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_HS.predict(X_test_HS)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
lr_HS.score(X_test_HS, Y_test_HS)

0.6967088607594937

#### All non-bias features

In [ ]:
# all parameters not specified are set to their defaults
lr_HS_all = LogisticRegression()

lr_HS_all.fit(X_train, Y_train_HS)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_HS_all.predict(X_train)

array([1, 0, 0, ..., 1, 1, 0])

In [ ]:
lr_HS_all.score(X_test, Y_test_HS)

0.7478481012658228

## Abusive

### Logistic Regression

#### Abusive features

In [ ]:
# all parameters not specified are set to their defaults
lr_A = LogisticRegression()

lr_A.fit(X_train_A, Y_train_A)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_A.predict(X_test_A)

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
lr_A.score(X_test_A, Y_test_A)

0.789113924050633

#### All non-bias features

In [ ]:
# all parameters not specified are set to their defaults
lr_A_all = LogisticRegression()

lr_A_all.fit(X_train, Y_train_A)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
lr_A_all.predict(X_test)

array([0, 0, 0, ..., 1, 0, 1])

In [ ]:
78 85lr_A_all.score(X_test, Y_test_A)

0.8556962025316456